# A review of probability theory

Say we need to model flipping a coin. We might express it with:

$P(x=\text{heads})=0.5$

$P(x=\text{tails})=0.5$

The discrete random variable $x$ denotes the possible outcomes, which is either heads or tails in this scenario.

The function $P(\cdot)$ outputs the probability of the outcome.

In flipping coins, the events are independent, which means each flip of a coin does not affect the probability of any future flip.

## Conditional Probability

However, drawing a card from a deck does affect the next draw.

What is the probability of drawing a King from a deck of poker cards?

$P(x_0=\text{any King})=4/52=1/13$

What is the probability of drawing a King after already drawing a King of Heart?

$P(x_1=\text{any King}|x_0=\text{King of Heart})=3/51$, where $x_0$ denotes the first draw and $x_1$ denotes the second draw

Drawing a king of heart affects all future draws. This is modeled using conditional probability. 

$P(x_1|x_0)$ is the probability of $x_1$ happening given $x_0$ already happened.

Note that $P(x_1|x_0)$ also implies that any other random variable other than $x_0$ has no impact on the probability of $x_1$.

Meaning $x_1$ is dependent on $x_0$, and $x_1$ is independent from any other random variables.


# Language model
One of the most influential idea from statistical NLP is the idea to model language with conditional probability. This idea extends even to neural NLP.

In NLP, we want a way to evaluate whether a given sentence is a valid sentence.

What is the probability of the sentence "An apple is sweet." is a valid sentence?

What is the probability of the sentence "apple sweet An is." is a valid sentence?

While it is unclear what the exact value should be, most people would agree $P(\text{"apple sweet An is."})$ should be a lot lower than $P(\text{"An apple is sweet."})$.

There are different ways to assign/calculate probability: [frequentist probability](https://en.wikipedia.org/wiki/Frequentist_probability), [Bayesian probability](https://en.wikipedia.org/wiki/Bayesian_probability), [classical definition of probability](https://en.wikipedia.org/wiki/Classical_definition_of_probability).

We will be using the frequentist probability:
$$P(x) \approx \frac{n_x}{n_t}$$

Where $n_x$ is the number of outcomes which $x$ happened, and $n_t$ is the total number of trials.

Under this view, the probability of the sentence "An apple is sweet." is a valid sentence might be approximated by:

$$P(\text{"An apple is sweet."}) \approx \frac{\text{# of "An apple is sweet." sentence}}{\text{# of sentence in existence}}$$

However, this may not be a good idea. Just because no one has written a particular sentence does not mean it is not a valid sentence.

Maybe no one ever wrote the sentence: "Bart walked down to Kwik-E-Mart and bought an apple from Apu.", which would mean it gets a probability of 0. But that does not mean it is not a valid sentence that some one might write.

Clearly, a smarter way is needed. 


## Language model: unigram
One idea is to break down the probability of the sentence into probability of the words.

$P(x_0 = \text{An}, x_1 = \text{apple}, x_2 = \text{is}, x_3 = \text{sweet}) = P(x_i = \text{An})P(x_i = \text{apple})P(x_i = \text{A})P(x_i = \text{is})P(x_i = \text{sweet})$

This is usually written in a more compact form in NLP:

$P(\text{An apple is sweet}) = P(\text{An})P(\text{apple})P(\text{is})P(\text{sweet})$

The above formula assumes the probability of each word is independent. Meaning it does not consider the context that the word occur in.

### Implementing unigram

We will be using the newsgroup dataset. We will break down each post into sentences then tokenize each word and lemmatize the tokens. We are also going to convert all uppercase to lowercase. This is to reduce variation for the same word.

Under frequentist probability we could assign the probability of the word "apple" to:

$$P(apple) \approx \frac{\text{# of "apple"}}{\text{# of words}}$$

So we will need a count of the words. 


In [1]:
import math
from collections import defaultdict

import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(data_home='data', remove=('headers', 'footers', 'quotes'))

nltk.data.path.append('data')
nltk.download('punkt', download_dir='data')
nltk.download('wordnet', download_dir='data')
lemmatizer = WordNetLemmatizer() 

num_post = 100  # Only process this many posts. Saves time.

[nltk_data] Downloading package punkt to data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to data...
[nltk_data]   Package wordnet is already up-to-date!


Just like spaCy, nltk is NLP package for python. 

It features much more algorithm that spaCy, but it is less optimized. 

It was developed for statistical NLP.

In [2]:
total_count = 0
word_count = defaultdict(int)
for post in dataset.data[:num_post]:
    sents = nltk.sent_tokenize(post)
    for sent in sents:
        tokens = nltk.word_tokenize(sent.lower())
        for token in tokens:
            word_count[lemmatizer.lemmatize(token)] += 1
            total_count += 1

Now that we have counted the words, we have a probability for each word in our data.

What is the probability for the word "if"?

In [3]:
word = "if"

word_count[word] / total_count  # The probability of the word according to our formula

0.002851033499643621

$P(\text{if})$ is 0.285% according to the model we have trained with the data given.

What is the probability of the sentence: "What is that car?"?

In [4]:
sent = "What is that car?"

prob = 1
for token in nltk.word_tokenize(sent.lower()):
    unigram_prob = word_count[lemmatizer.lemmatize(token)] / total_count
    prob *= unigram_prob

print(f"Probability of \"{sent}\": {prob}")

Probability of "What is that car?": 1.1210876923308169e-12


### Log probability

The probability of that sentence is quite small. For some sentence, the probability might be so small as to underflow. Therefore, log probability is used for numerical stability reasons.

$\text{log}(x \cdot y) = \text{log}(x) + \text{log}(y)$

Instead of multiplying probabilities, we will be adding log probabilities.

In [5]:
sent = "What is that car?"

prob = 0
for token in nltk.word_tokenize(sent.lower()):
    unigram_prob = math.log(word_count[lemmatizer.lemmatize(token)] / total_count)
    prob += unigram_prob

print(f"Log probability of \"{sent}\": {prob}")

Log probability of "What is that car?": -27.51672174801957


### Smoothing

What is the probability of the word "jupyter"?

In [6]:
word = "jupyter"

word_count[word] / total_count 

0.0

The probability should be 0. As our model never saw the word "jupyter". 

In other words, any sentence with an unknown word to the model has a probability of 0. This is undesirable as a single unknown word should not ruin an otherwise good sentence.

To deal with unknown words, smoothing is used. There are a variety of smoothing techniques. We will be using add-one smoothing:

$$P(x) = \frac{\text{# of }x + 1}{\text{# of words} + \text{vocab size}}$$

The # of words refers to how many words in the dataset.

The vocab size refers to how many types of word in the dataset.

For example, if the whole dataset is: "apple apple. orange orange."

\# of words: 4

vocab size: 2

With add-one smoothing what is the probability of "if"?

In [7]:
word = "if"
vocab_size = len(word_count)

(word_count[word] + 1) / (total_count + vocab_size)

0.002407704654895666

Now the $P(\text{if})$ becomes 0.241%. 

Note that this is slightly smaller than without add-one smoothing. What add-one smoothing and other smoothing are doing is taking a little bit of probability from known words and giving it to the unknown word category.

All unknown words are given a fixed probability of $$\frac{1}{\text{# of words} + \text{vocab size}}$$

#### Dealing with unknown without smoothing
It is also possible give unknown words a fix probability of $\frac{1}{\text{vocab size}}$ without removing probability from other words. 

Technically, it would not be a probability as it no longer adds up to 1, but it works well in practice. 

### Practice:

#### Exercise 1:
What is the probability of "What is that car?" with add-one smoothing?


In [8]:
# code here

Probability of "What is that car?": 4.821918666285085e-13


#### Exercise 2:
What is the log probability of "What is that car?" with add-one smoothing?

In [9]:
# code here

Log probability of "What is that car?": -28.360259822125837

### Problem of unigram
It was briefly mentioned that unigram doesn't consider the context of the word. It should be apparent now.

The probability of of "are" in "There are an apple." and "There are apples." are the same. Even "There are an apple" is grammatically incorrect.

Both are equal to $\frac{\text{# of "are"}}{\text{# of words}}$ when without smoothing.

There needs to be a way to consider the context of the given word.

## Language model: n-gram

Under the n-gram model, the probability of a word given the $n-1$ word(s) that comes before it.

So a bigram ($n=2$) would look something like: 

$P(\text{apple}|\text{an})$

A trigram ($n=3$) would look something like: 

$P(\text{is}|\text{an apple})$

Which is the probability of the word "is" appearing given two word before it is "a apple".

So the probability of $P(\text{is}|\text{an apple})$ should be a lot higher than $P(\text{are}|\text{an apple})$.

### Bigram

For the $n=2$ case, the probability is written as:

$P(w_i|w_{i-1})$

Where $w_i$ is the word that follows $w_{i-1}$.

Example: In "an apple", $w_i$ would be "apple", and $w_{i-1}$ would be "an".

Under the bigram model, we are assuming that the probability of $w_i$ only depends on $w_{i-1}$.

To calculate the probability of a sentence using bigram (conditional probability), we need chain rule.

#### Chain rule

$$P(X_n , \dots , X_1) = P(X_n | X_{n-1} , \dots , X_1) P(X_{n-1} , \dots , X_1)$$

So for 4 word sentence:
$$P(w_0 w_1 w_2 w_3) = P(w_3|w_0 w_1 w_2) P(w_0 w_1 w_2) = P(w_3 | w_0 w_1 w_2) P(w_2 | w_0 w_1) P(w_0 w_1) = P(w_3 | w_0 w_1 w_2) P(w_2 | w_0 w_1) P(w_1 | w_0) P(w_0)$$

In bigram, we are assuming each word is independent from every word except the word before it. So we can cancel all other words.

$$P(w_0 w_1 w_2 w_3) = P(w_3|w_2) P(w_2|w_1)P(w_1|w_0)P(w_0) = P(w_0)P(w_1|w_0)P(w_2|w_1)P(w_3|w_2)$$
 

The probability of the sentence "An apple is sweet" with bigram becomes:

$$P(\text{An apple is sweet}) = P(\text{$<$S$>$}) P(\text{An} | \text{$<$S$>$})P(\text{apple} | \text{An})P(\text{is} | \text{apple})P(\text{sweet} | \text{is})$$

Where $<$S$>$ denotes a special start symbol. Since there is no word that comes before "An". We will define $P($<$S$>$)$ as 1.

### Implementing bigram

To calculate conditional probability, we will use the following formula:

$$P(A|B)=\frac{P(A, B)}{P(B)}$$

For bigram that would be $$P(w_i|w_{i-1}) = \frac{P(w_{i-1} w_i)}{P(w_{i-1})} \approx \frac{C(w_{i-1} w_i)}{C(w_{i-1})}$$

Where $C( \cdot )$ is the count function.

So for $w_{i-1} = \text{a}$, $w_i = \text{apple}$:

$$P(\text{apple} | \text{an}) \approx \frac{\text{# of "an apple"}}{\text{# of "an"}}$$

So to calculate bigram, we need count of individual words and word pairs.

In [10]:
START_TOKEN = "<S>"

total_count = 0
word_count = defaultdict(int)
pair_count = defaultdict(int)
for post in dataset.data[:num_post]:
    sents = nltk.sent_tokenize(post)
    for sent in sents:
        tokens = nltk.word_tokenize(sent.lower())
        last_word = START_TOKEN
        for token in tokens:
            word = lemmatizer.lemmatize(token)
            pair_count[(last_word, word)] += 1       
            word_count[word] += 1
            total_count += 1
            last_word = word

Once the words and word pairs are counted we can calculate the bigram probability.

What is the probability of "is" after the word "what"?

In [11]:
words = ("what", "is")

pair_count[words] / word_count[words[0]]

0.04878048780487805

### Backoff
It is possible that a there is a unseen bigram, but the unigram is seen.

In this case, we can use the unigram instead of bigram

Say we need to evaluate "day" in "good day":

In [12]:
words = ("good", "day")

pair_count[words] / word_count[words[0]]

0.0

There is no such bigram in the data, so the probability is 0. In that case, we can use the unigram "day" to replace it. However, we need to weight it first.

The easiest yet effective method is stupid back-off, which weight the probability by 0.4 each time we back off. 

So by going from bigram to unigram we multiple the unigram probability by 0.4.

What is the probability of "day" in "good day" with stupid back-off from bigram to unigram?

In [13]:
words = ("good", "day")

prob = pair_count[words] / word_count[words[0]]
if prob == 0:
    prob = 0.4 * word_count[words[1]] / total_count
    
print(f"Probability: {prob}")

Probability: 0.00033541570584042597


The result from stupid back-off is not a true probability since all the outcomes no longer adds up to 1. There are back-offs which output true probabilities such as Katz's back-off.

### Practice:

#### Exercise 3:
Train a trigram model.

hint: $P(w_i|w_{i-2}w_{i-1}) = \frac{C(w_{i-2}w_{i-1}w_i)}{C(w_{i-2}w_{i-1})}$


In [14]:
# code here

#### Exercise 4:
What is the probability of "a" after "it is"?

In [15]:
# code here

## Solutions:

In [16]:
# Solution to exercise 1
sent = "What is that car?"
vocab_size = len(word_count)

prob = 1
for token in nltk.word_tokenize(sent.lower()):
    unigram_prob = (word_count[lemmatizer.lemmatize(token)] + 1) / (total_count + vocab_size)
    prob *= unigram_prob

print(f"Probability of \"{sent}\": {prob}")

Probability of "What is that car?": 4.822760041028505e-13


In [17]:
# Solution to exercise 2
sent = "What is that car?"
vocab_size = len(word_count)

prob = 0
for token in nltk.word_tokenize(sent.lower()):
    unigram_prob = math.log((word_count[lemmatizer.lemmatize(token)] + 1) / (total_count + vocab_size))
    prob += unigram_prob

print(f"Log probability of \"{sent}\": {prob}")

Log probability of "What is that car?": -28.360259822125837


In [18]:
# Solution to exercise 3
START_TOKEN = "<S>"

context_count = defaultdict(int)
phrase_count = defaultdict(int)
for post in dataset.data[:num_post]:
    sents = nltk.sent_tokenize(post)
    for sent in sents:
        tokens = nltk.word_tokenize(sent.lower())
        second_last_word = START_TOKEN
        last_word = START_TOKEN
        for token in tokens:
            word = lemmatizer.lemmatize(token)
            phrase_count[(second_last_word, last_word, word)] += 1       
            context_count[(second_last_word, last_word)] += 1 
            second_last_word = last_word
            last_word = word

In [19]:
# Solution to exercise 4
words = ("it", "is", "a")

phrase_count[words] / context_count[(words[0], words[1])]

0.07407407407407407